# Descarga de datos de Fallecimientos desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime
import seaborn as sns
%matplotlib inline

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [2]:
datadir='datos_sanidad/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

# Actualización día ZERO con el histórico

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [3]:
def descarga(url,num):
    print('Descargando:', url)
    fn=datadir+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [4]:
for i in range(77,263):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_77_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_78_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_79_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_80_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_81_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_82_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_83_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/aler

KeyboardInterrupt: 

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [5]:
descarga(URL_reg.format(265),265)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf


True

### Función para extraer el texto de la página correspondiente del informe pdf

In [6]:
# Extract PDF text using PDFMiner. Adapted from
# http://stackoverflow.com/questions/5725278/python-help-using-pdfminer-as-a-library

def pdf_to_text(pdfname, pagenum=None):

    # PDFMiner boilerplate
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    laparams = LAParams()
    device = None
    try:
        device = TextConverter(rsrcmgr, sio, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Extract text
        with open(pdfname, 'rb') as fp:
            for i, page in enumerate(PDFPage.get_pages(fp)):
                if pagenum is None or pagenum == i:
                    interpreter.process_page(page)

        # Get text from StringIO
        text = sio.getvalue()
    finally:
        # Cleanup
        sio.close()
        if device is not None:
            device.close()

    return text

### Creamos el pandas datos, al que vamos incorporando los datos leidos

In [7]:
datos=pd.DataFrame()

### Descarga del documento 235 a la actualidad

In [8]:
pattern=re.compile(r'(\n{1,2}\d*[,.]?\d+¥? ){19}')
pattern=re.compile(r'(\n\nTotal) ((\n{1,2}\d*[,.]?\d+¥? ){19})')
for i in range(235,266):
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    text = pdf_to_text(fn, pagenum=4)  #con que pagina queremos trabajar?
    cadena=re.search(pattern,text).group(2)         
    print ('Documento {}-->'.format(i),cadena.replace('¥','').replace('\n','').split())
    datos.loc[:,str(i)]=cadena.replace('¥','').replace('\n','').split()

Documento 235--> ['2.270', '1566', '391', '352', '267', '248', '3.338', '3.446', '5.960', '13', '1.719', '676', '859', '10.211', '11', '283', '668', '2.032', '442']
Documento 236--> ['2.314', '1598', '409', '356', '270', '248', '3.349', '3.502', '5.961', '16', '1.727', '687', '882', '10.247', '11', '288', '675', '2.038', '453']
Documento 237--> ['2.370', '1613', '414', '357', '273', '250', '3.367', '3.528', '5.968', '16', '1.737', '697', '884', '10.327', '11', '299', '687', '2.046', '454']
Documento 238--> ['2.402', '1620', '419', '357', '277', '251', '3.383', '3.549', '5.972', '16', '1.751', '701', '895', '10.350', '12', '300', '694', '2.061', '456']
Documento 239--> ['2.432', '1627', '427', '357', '278', '252', '3.405', '3.567', '5.977', '18', '1.762', '703', '895', '10.403', '12', '306', '699', '2.061', '458']
Documento 240--> ['2.470', '1654', '437', '362', '282', '252', '3.425', '3.599', '5.991', '19', '1.773', '714', '904', '10.419', '12', '317', '703', '2.082', '463']
Documento 

### Aquí tenemos una serie de días que han dado error en el escrapeo y no pude solucionar, luego introducimos los valores manualmente:

In [9]:
dia_234=[2183,1559,383,344,266,248,3336,3421,5958,13,1709,670,850,10155,11,279,666,2029,441] 

In [10]:
dia_137=[1404,826,314,209,151,202,2945,1928,5587,4,1332,508,609,8691,2,148,490,1424,362]

In [11]:
datos.loc[:,'234']=dia_234

In [12]:
datos.loc[:,'137']=dia_137

### Ahora metemos la serie antígua..del 77 al 234

In [13]:
texto1="Total"

texto2="\n\n"

for i in range(77,234): #original 100 a 234
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    text = pdf_to_text(fn, pagenum=1)
    #with open(fn1, 'w', encoding='utf-8') as fp:
    #with open(fn1, 'w') as fp:    
    #        fp.write(page1)
 

    #with open(fn1) as fp:
    #    text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
    
    if ((((i >= 122) & (i<=139))) & (i!=137)):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[113])
        #print (i,'ojo',' ->',lista[113:113+19])
        datos.loc[:,str(i)]=lista[113:113+19]
            
    elif (i==151):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[127])
        datos.loc[:,str(i)]=lista[127:127+19]
    elif (i==154):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[123])
        datos.loc[:,str(i)]=lista[123:123+19]
        
    elif (i!=137):
        lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[1])
        datos.loc[:,str(i)]=lista[1:1+19]

        #print (i,' -> ',lista[0],'-->',lista[1:20])

77  ->  912
78  ->  940
79  ->  967
80  ->  993
81  ->  1.013
82  ->  1.017
83  ->  1.050
84  ->  1.079
85  ->  1.107
86  ->  1.131
87  ->  1.145
88  ->  1.157
89  ->  1.168
90  ->  1.188
91  ->  1.207
92  ->  1.238
93  ->  1.253
94  ->  1.256
95  ->  1.263
96  ->  1.267
97  ->  1.281
98  ->  1.294
99  ->  1.301
100  ->  1.317
101  ->  1.320
102  ->  1.322
103  ->  1.326
104  ->  1.332
105  ->  1.336
106  ->  1.344
107  ->  1.355
108  ->  1.358
109  ->  1.358
110  ->  1.358
111  ->  1.371
112  ->  1.375
113  ->  1.377
114  ->  1.389
115  ->  1.391
116  ->  1.334
117  ->  1.404
118  ->  1.404
119  ->  1.404
120  ->  1.404
121  ->  1.404
122  ->  1.404
123  ->  1.404
124  ->  1.404
125  ->  1.404
126  ->  1.404
127  ->  1.404
128  ->  1.404
129  ->  1.404
130  ->  1.404
131  ->  1.404
132  ->  1.404
133  ->  1.404
134  ->  1.404
135  ->  1.404
136  ->  1.404
138  ->  1.404
139  ->  1.404
140  ->  1.404
141  ->  1.426
142  ->  1.426
143  ->  1.426
144  ->  1.426
145  ->  1.426
146  ->  1.

### Filtramos las columnas en las fechas correctas: desde 77 hasta hoy, y formateamos el index con la denominación correcta de la CA

In [14]:
datos

,235,236,237,238,239,240,241,242,243,244,...,224,225,226,227,228,229,230,231,232,233
0,2.270,2.314,2.370,2.402,2.432,2.470,2.570,2.602,2.664,2.695,...,1.965,1.979,2.020,2.020,2.053,2.065,2.094,2.137,2.176,2.182
1,1566,1598,1613,1620,1627,1654,1691,1706,1730,1753,...,1412,1470,1481,1485,1495,1501,1508,1528,1541,1543
2,391,409,414,419,427,437,466,484,497,520,...,351,354,356,357,361,364,365,371,371,377
3,352,356,357,357,357,362,363,364,371,372,...,313,316,316,316,321,325,327,327,332,336
4,267,270,273,277,278,282,286,289,294,295,...,240,241,247,248,250,250,251,257,259,265
5,248,248,250,251,252,252,255,255,254,257,...,237,237,237,240,240,240,242,247,247,247
6,3.338,3.349,3.367,3.383,3.405,3.425,3.437,3.445,3.414,3.438,...,3.244,3.254,3.266,3.267,3.277,3.304,3.322,3.330,3.331,3.333
7,3.446,3.502,3.528,3.549,3.567,3.599,3.659,3.673,3.757,3.810,...,3.179,3.199,3.232,3.241,3.280,3.297,3.307,3.346,3.380,3.400
8,5.960,5.961,5.968,5.972,5.977,5.991,6.001,6.036,7.073,7.147,...,5.865,5.875,5.883,5.883,5.890,5.895,5.909,5.913,5.919,5.927
9,13,16,16,16,18,19,23,26,29,29,...,11,11,12,12,12,12,12,13,13,13


In [15]:
datos=datos[[str(i) for i in range(77,266)]]

In [16]:
datos.index=['Andalucia','Aragon','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
            'Castilla y Leon','Cataluña','Ceuta','C.Valenciana','Extremadura','Galicia','Madrid','Melilla','Murcia',
             'Navarra','Pais Vasco','La Rioja']

In [17]:
datos=datos.applymap(lambda x: int(str(x).replace(".","")))

necesitamos un fichero adicional en el que se relacionen los nº de orden de los docs con las fechas en las que se publicaron

In [18]:
claves=pd.read_excel(datadir+'clave_numero_fecha.xlsx')

In [19]:
claves

,Codigo,Fecha
0,77,2020-04-16
1,78,2020-04-17
2,79,2020-04-18
3,80,2020-04-19
4,81,2020-04-20
...,...,...
184,261,2020-11-30
185,262,2020-12-01
186,263,2020-12-02
187,264,2020-12-03


In [20]:
#claves=claves.loc[claves.index[:]]

In [21]:
datos.columns=claves['Fecha']

Finalmente guardamos el `pandas`en un fichero csv:

In [22]:
datos.to_csv('datos_sanidad_matriz.csv')

Preparamos también una versión en formato tabla:

In [23]:
datos_tabla=datos.unstack().reset_index()[['level_1','Fecha',0]]

In [24]:
datos_tabla.columns=['Comunidad','Fecha','Fallecidos']

que guardamos también en el directorio local:

In [25]:
datos_tabla.to_csv('datos_sanidad_tabla.csv')